An Example – Scheduling Example Extended
In our last example, we explored the scheduling of 2 factories.

Both factories had 2 costs:

Fixed Costs – Costs incurred while the factory is running
Variable Costs – Cost per unit of production
We’re going to introduce a third cost – Start up cost.

This will be a cost incurred by turning on the machines at one of the factories.

In this example, our start-up costs will be:

Factory A – €20,000

Factory B – €400,000

Let’s start by reminding ourselves of the input data.

In [2]:
import pandas as pd
import pulp

factories = pd.read_csv('factory_variables.csv', index_col=['Month', 'Factory'])
factories

Max_Capacity  Min_Capacity  Variable_Costs  Fixed_Costs
Month Factory                                                         
1     A              100000         20000              10          500
      B               50000         20000               5          600
2     A              110000         20000              11          500
      B               55000         20000               4          600
3     A              120000         20000              12          500
      B               60000         20000               3          600
4     A              145000         20000               9          500
      B              100000         20000               5          600
5     A              160000         20000               8          500
      B                   0             0               0            0
6     A              140000         20000               8          500
      B               70000         20000               6          600
7     A              155000         20000               5          500
      B               60000         20000               4          600
8     A              200000         20000               7          500
      B              100000         20000               6          600
9     A              210000         20000               9          500
      B              100000         20000               8          600
10    A              197000         20000              10          500
      B              100000         20000              11          600
11    A               80000         20000               8          500
      B              120000         20000              10          600
12    A              150000         20000               8          500
      B              150000         20000              12          600

In [3]:
demand = pd.read_csv('monthly_demand.csv', index_col=['Month'])
demand

,Demand
Month,
1,120000
2,100000
3,130000
4,130000
5,140000
6,130000
7,150000
8,170000
9,200000


We’ll begin by defining our decision variables, we have an additional binary variable for switching on the factory.

In [4]:
# Production
production = pulp.LpVariable.dicts("production",
                                     ((month, factory) for month, factory in factories.index),
                                     lowBound=0,
                                     cat='Integer')

# Factory Status, On or Off
factory_status = pulp.LpVariable.dicts("factory_status",
                                     ((month, factory) for month, factory in factories.index),
                                     cat='Binary')

# Factory switch on or off
switch_on = pulp.LpVariable.dicts("switch_on",
                                    ((month, factory) for month, factory in factories.index),
                                    cat='Binary')

We instantiate our model and define our objective function, including start up costs

In [5]:
# Instantiate the model
model = pulp.LpProblem("Cost minimising scheduling problem", pulp.LpMinimize)

# Select index on factory A or B
factory_A_index = [tpl for tpl in factories.index if tpl[1] == 'A']
factory_B_index = [tpl for tpl in factories.index if tpl[1] == 'B']

# Define objective function
model += pulp.lpSum(
    [production[m, f] * factories.loc[(m, f), 'Variable_Costs'] for m, f in factories.index]
    + [factory_status[m, f] * factories.loc[(m, f), 'Fixed_Costs'] for m, f in factories.index]
    + [switch_on[m, f] * 20000 for m, f in factory_A_index]
    + [switch_on[m, f] * 400000 for m, f in factory_B_index]
)

C:\Users\dojin\Anaconda3\lib\site-packages\pulp\pulp.py:1114: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")


Now we begin to build up our constraints as in Part 5

In [13]:
# Production in any month must be equal to demand
months = demand.index
for month in months:
    model += production[(month, 'A')] + production[(month, 'B')] == demand.loc[month, 'Demand']

# Production in any month must be between minimum and maximum capacity, or zero.
for month, factory in factories.index:
    min_production = factories.loc[(month, factory), 'Min_Capacity']
    max_production = factories.loc[(month, factory), 'Max_Capacity']
    model += production[(month, factory)] >= min_production * factory_status[month, factory]
    model += production[(month, factory)] <= max_production * factory_status[month, factory]

# Factory B is off in May
model += factory_status[5, 'B'] == 0
model += production[5, 'B'] == 0

But now we want to add in our constraints for switching on.

A factory switches on if:

It is off in the previous month (m-1)

AND it on in the current month (m).

As we don’t know if the factory is on before month 0, we’ll assume that the factory has switched on if it is on in month 1.

In [14]:
for month, factory in factories.index:
    # In month 1, if the factory ison, we assume it turned on
    if month == 1:
        model += switch_on[month, factory] == factory_status[month, factory]
    
    # In other months, if the factory is on in the current month AND off in the previous month, switch on = 1
    else:
        model += switch_on[month, factory] >= factory_status[month, factory] - factory_status[month-1, factory]
        model += switch_on[month, factory] <= 1 - factory_status[month-1, factory]
        model += switch_on[month, factory] <= factory_status[month, factory]
        

We’ll then solve our model

In [15]:
model.solve()
pulp.LpStatus[model.status]

'Optimal'

In [16]:
output = []
for month, factory in production:
    var_output = {
        'Month': month,
        'Factory': factory,
        'Production': production[(month, factory)].varValue,
        'Factory Status': factory_status[(month, factory)].varValue,
        'Switch On': switch_on[(month, factory)].varValue
    }
    output.append(var_output)
output_df = pd.DataFrame.from_records(output).sort_values(['Month', 'Factory'])
output_df.set_index(['Month', 'Factory'], inplace=True)
output_df

Production  Factory Status  Switch On
Month Factory                                       
1     A           70000.0             1.0        1.0
      B           50000.0             1.0        1.0
2     A           45000.0             1.0        0.0
      B           55000.0             1.0        0.0
3     A           70000.0             1.0        0.0
      B           60000.0             1.0        0.0
4     A           30000.0             1.0        0.0
      B          100000.0             1.0        0.0
5     A          140000.0             1.0        0.0
      B               0.0             0.0        0.0
6     A           60000.0             1.0        0.0
      B           70000.0             1.0        1.0
7     A           90000.0             1.0        0.0
      B           60000.0             1.0        0.0
8     A           70000.0             1.0        0.0
      B          100000.0             1.0        0.0
9     A          100000.0             1.0        0.0
      B          100000.0             1.0        0.0
10    A          170000.0             1.0        0.0
      B           20000.0             1.0        0.0
11    A           80000.0             1.0        0.0
      B           60000.0             1.0        0.0
12    A          100000.0             1.0        0.0
      B               0.0             0.0        0.0

Interestingly, we see that it now makes economic sense to keep factory B on after it turns off in month 5 up until month 12.

Previously, we had the case that it was not economic to run factory B in month 10, but as there is now a significant cost to switching off and back on, the factory runs through month 10 at its lowest capacity (20,000 units).



In [ ]:
# For those interested in using my function defined above (make_io_and_constraint). Instead of:

# model += switch_on[month, factory] >= factory_status[month, factory] - factory_status[month-1, factory]
# model += switch_on[month, factory] <= 1 - factory_status[month-1, factory]
# model += switch_on[month, factory] <= factory_status[month, factory]

In [7]:
def make_io_and_constraint(y1, x1, x2, target_x1, target_x2):
    """
    Returns a list of constraints for a linear programming model
    that will constrain y1 to 1 when
    x1 = target_x1 and x2 = target_x2; 
    where target_x1 and target_x2 are 1 or 0
    """
    binary = [0,1]
    assert target_x1 in binary
    assert target_x2 in binary
    
    if x1 == 1 and x2 == 1:
        return [
            y1 >= x1 + x2 - 1,
            y1 <= x1,
            y1 <= x2
        ]
    elif x1 == 1 and x2 == 0:
        return [
            y1 >= x1 - x2,
            y1 <= x1,
            y1 <= (1 - x2)
        ]
    elif x1 == 0 and x2 == 1:
        return [
            y1 >= x2 - x1,
            y1 <= (1 - x1),
            y1 <= x2
        ]
    else:
        return [
            y1 >= - (x1 + x2 -1),
            y1 <= (1 - x1),
            y1 <= (1 - x2)
        ]

In [9]:
for constraint in make_io_and_constraint(switch_on[month, factory], 
                                        factory_status[month, factory], 
                                        factory_status[month-1, factory], 0, 1):
    model += constraint

In [17]:
# Print our objective function value (Total Costs)
pulp.value(model.objective)

13747000.0